# Generate Data

In [16]:
import numpy as np
import pandas as pd
import math
from doubleml.datasets import make_plr_CCDDHNR2018, make_plr_turrell2018
np.random.seed(1234)
n_rep = 1
n_obs = 10000
n_vars = 10
alpha = 0.5
data = list()
from sklearn.datasets import make_spd_matrix

def g(x):
    return np.exp(x)

def m(x):
    return np.sin(x)

theta = alpha = 0.5 
b = [1/k for k in range(1,n_vars+1)] # x weights 
sigma = make_spd_matrix(n_vars, random_state=42)

for i_rep in range(n_rep):
    #(x, y, d) = make_plr_turrell2018(alpha=alpha, n_obs=n_obs, dim_x=n_vars, return_type='array')
    df = pd.read_csv("/Users/pranjal/Desktop/Causal-Inference/data/penn_jae.dat", sep = ' ')

    outcome = 'inuidur1'
    treatment = 'tg'
    rest = list(df.drop([outcome, treatment, 'inuidur2','muld', 'Unnamed: 24', 'Unnamed: 25'], axis = 1).columns)
    rest = ['female', 'black', 'othrace',
       'dep', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 
       'agelt35', 'agegt54', 'durable', 'lusd', 'husd']
    df = df[[outcome] + [treatment] + rest]
    df = df.dropna()
    #df = df.fillna(0)
    #df = df[df.avg <= 15]
    y = df[outcome]
    d = df[treatment]
    x = df[rest].astype('float')
    print(y.shape, x.shape, d.shape)
    data.append((x, y, d))

(13913,) (13913, 15) (13913,)


In [17]:
df.columns

Index(['inuidur1', 'tg', 'female', 'black', 'othrace', 'dep', 'q1', 'q2', 'q3',
       'q4', 'q5', 'q6', 'agelt35', 'agegt54', 'durable', 'lusd', 'husd'],
      dtype='object')

In [18]:
female+black+othrace+ dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd

NameError: name 'female' is not defined

# Naive ML
- no orthogonalisation, no crossfitting

In [19]:
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, y - g_hat)
    return psi_a, psi_b

In [20]:
ml_m.fit(x,d).predict_proba(x)

AttributeError: 'RandomForestRegressor' object has no attribute 'predict_proba'

In [ ]:
from doubleml import DoubleMLData
from doubleml import DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
face_colors = sns.color_palette('pastel')
edge_colors = sns.color_palette('dark')
np.random.seed(1111)
ml_l = RandomForestRegressor()
ml_m = RandomForestRegressor()
#ml_m = LogisticRegression()
ml_g = clone(ml_l)

# to speed up the illustration we hard-code the simulation results
theta_nonorth = np.empty(n_rep)
se_nonorth = np.empty(n_rep)
t_nonorth = np.empty(n_rep)
p_nonorth = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
#for i_rep in range(1):
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y.ravel(), d.ravel())
    print(obj_dml_data)
    obj_dml_plr_nonorth = DoubleMLPLR(obj_dml_data,
                                      ml_l, ml_m, ml_g,
                                      n_folds=2,
                                      apply_cross_fitting=False,
                                      score=non_orth_score)
    obj_dml_plr_nonorth.fit()
    theta_nonorth[i_rep] = obj_dml_plr_nonorth.coef[0]
    se_nonorth[i_rep] = obj_dml_plr_nonorth.se[0]

# Orthogonal Machine Learning
- resolves regularization bias, but not overfitting

In [ ]:
import numpy as np
np.random.seed(2222)
# to speed up the illustration we hard-code the simulation results
theta_orth_nosplit = np.empty(n_rep)
se_orth_nosplit = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
#for i_rep in range(1):
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr_orth_nosplit = DoubleMLPLR(obj_dml_data,
                                           ml_l, ml_m, ml_g,
                                           n_folds=1,
                                           score='IV-type',
                                           apply_cross_fitting=False)
    obj_dml_plr_orth_nosplit.fit()
    theta_orth_nosplit[i_rep] = obj_dml_plr_orth_nosplit.coef[0]
    se_orth_nosplit[i_rep] = obj_dml_plr_orth_nosplit.se[0]

# Orthogonal ML + Cross fitting (DML)

In [21]:
np.random.seed(42)
# to speed up the illustration we hard-code the simulation results
theta_dml = np.empty(n_rep)
se_dml = np.empty(n_rep)

# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr = DoubleMLPLR(obj_dml_data,
                              ml_l, ml_m, ml_g,
                              n_folds=2,
                              score='IV-type')
    obj_dml_plr.fit()
    theta_dml[i_rep] = obj_dml_plr.coef[0]
    se_dml[i_rep] = obj_dml_plr.se[0]

0


# Regular OLS

In [22]:
np.random.seed(42)
import statsmodels.api as sm # for OLS 
# to speed up the illustration we hard-code the simulation results
theta_ols = np.empty(n_rep)
se_ols = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    (x, y, d) = data[i_rep]
    OLS = sm.OLS(y,sm.add_constant(np.c_[d,x]))
    results = OLS.fit()
    theta_ols[i_rep] = results.params[1]
    se_ols[i_rep] = results.bse[1]   

# Distribution of Theta

In [23]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]

table = PrettyTable()
table.field_names = ['est', 'coef', 'std','t','p','2.5%','97.25%']
a = ['OLS']+ np.c_[results.params[1], results.bse[1], results.tvalues[1], results.pvalues[1], np.nan, np.nan].reshape(-1).tolist()
table.add_row(a)
a = ['Naive-ML']+ np.array(obj_dml_plr_nonorth.summary).reshape(-1).tolist()
table.add_row(a)
a = ['Ortho-ML']+ np.array(obj_dml_plr_orth_nosplit.summary).reshape(-1).tolist()
table.add_row(a)
a = ['OrthoML+Crossfitting (DML)']+ np.array(obj_dml_plr.summary).reshape(-1).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+----------------------------+--------+-------+--------+-------+--------+--------+
|            est             |  coef  |  std  |   t    |   p   |  2.5%  | 97.25% |
+----------------------------+--------+-------+--------+-------+--------+--------+
|            OLS             | -0.091 | 0.044 | -2.050 | 0.040 |  nan   |  nan   |
|          Naive-ML          | -0.087 | 0.042 | -2.074 | 0.038 | -0.170 | -0.005 |
|          Ortho-ML          | -0.122 | 0.042 | -2.920 | 0.003 | -0.205 | -0.040 |
| OrthoML+Crossfitting (DML) | -0.062 | 0.047 | -1.318 | 0.188 | -0.154 | 0.030  |
+----------------------------+--------+-------+--------+-------+--------+--------+


# First Stage Checks

In [91]:
OLS = sm.OLS(y,sm.add_constant(np.c_[d, x]))
results = OLS.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     14.98
Date:                Sun, 11 Dec 2022   Prob (F-statistic):           1.52e-13
Time:                        17:14:09   Log-Likelihood:                -433.15
No. Observations:                 428   AIC:                             878.3
Df Residuals:                     422   BIC:                             902.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2190      0.300     -0.729      0.466      -0.809       0.371
x1             0.1096      0.014      7.600      0.000       0.081       0.138
x2             0.0155      0.005      3.261      0.001       0.006       0.025
x3            -0.0035      0.005     -0.666      0.505      -0.014       0.007
x4            -0.0755      0.089     -0.851      0.395      -0.250       0.099
x5            -0.0177      0.028     -0.632      0.528      -0.073       0.037
==============================================================================
Omnibus:                       79.021   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              299.257
Skew:                          -0.773   Prob(JB):                     1.04e-65
Kurtosis:                       6.793   Cond. No.                         433.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [92]:
OLS = sm.OLS(d,sm.add_constant(np.c_[x]))
results = OLS.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   educ   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     3.366
Date:                Sun, 11 Dec 2022   Prob (F-statistic):            0.00997
Time:                        17:14:12   Log-Likelihood:                -953.86
No. Observations:                 428   AIC:                             1918.
Df Residuals:                     423   BIC:                             1938.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.6328      0.766     17.806      0.000      12.128      15.138
x1            -0.0049      0.016     -0.305      0.761      -0.036       0.027
x2            -0.0166      0.018     -0.947      0.344      -0.051       0.018
x3             0.6948      0.297      2.337      0.020       0.111       1.279
x4            -0.2298      0.094     -2.456      0.014      -0.414      -0.046
==============================================================================
Omnibus:                        4.161   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.125   Jarque-Bera (JB):                4.351
Skew:                           0.141   Prob(JB):                        0.114
Kurtosis:                       3.406   Cond. No.                         318.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [93]:
obj_dml_plr_nonorth.summary()

TypeError: 'DataFrame' object is not callable